In [1]:
import sys
sys.path.append('../../..')

In [2]:
from cvlization.torch.training_pipeline.lm.mdgpt import MDGPTTrainingPipeline

2024-03-25 14:11:57.653439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 14:11:58.274313: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12/lib64
2024-03-25 14:11:58.274365: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12/lib64
2024-03-25 14:11:58.274370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.c

In [3]:
pipe = MDGPTTrainingPipeline(config=MDGPTTrainingPipeline.Config())

tokens per iteration will be: 32,768


/home/zzsi/miniconda2/envs/cv/lib/python3.8/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [4]:
import numpy as np
data = np.load("flying_mnist_tokens_32frames_train.npy")
data.shape

(1000, 8, 64, 64)

In [5]:
pipe.train_data = data.reshape(1000, -1).astype(np.int64)
pipe.val_data = pipe.train_data

In [6]:
data2 = data.reshape(1000, -1)

In [7]:
meshgrid_args = [np.arange(s) for s in data.shape[1:]]
positions = np.array(
                np.meshgrid(
                    *meshgrid_args,
                    indexing="ij"
                ),
            ).transpose(1, 2, 3, 0)

print(positions.shape)

positions = positions.reshape(-1, positions.shape[-1])
print(f"data shape: {data2.shape}")
print(f"positions shape: {positions.shape}")

(8, 64, 64, 3)
data shape: (1000, 32768)
positions shape: (32768, 3)


In [8]:
positions

array([[ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0,  2],
       ...,
       [ 7, 63, 61],
       [ 7, 63, 62],
       [ 7, 63, 63]])

In [9]:
import torch
end_irow = torch.randint(data2.shape[0], (8,))
end_ix = torch.randint(data2.shape[1], (8,))
end_ix
block_size = 64 * 64 * 3
start_ix = end_ix - block_size

In [10]:
IGNORE_TOKEN = 5122

In [11]:
start_ix
dst_arr = np.ones((len(start_ix), block_size), dtype=np.int64) * IGNORE_TOKEN
for i in range(len(start_ix)):
    src_start = max(0, start_ix[i])
    src_end = end_ix[i]
    dst_start = max(0, -start_ix[i])
    dst_end = block_size
    # print(i, end_irow[i], src_start, src_end, dst_start, dst_end)
    dst_arr[i, dst_start:dst_end] = data2[end_irow[i], src_start:src_end]
dst_arr
x = dst_arr

In [30]:
x.shape, x

(torch.Size([8, 12288]),
 tensor([[5122, 5122, 5122,  ..., 4770, 4770, 4770],
         [4770, 4770, 4770,  ..., 4770, 4770, 4770],
         [4770, 4770, 4770,  ..., 4770, 4770, 4770],
         ...,
         [4770, 4770, 4770,  ..., 4770, 4770, 4770],
         [2352, 2352, 2352,  ..., 4770, 4770, 4770],
         [4770, 4770, 4770,  ..., 4770, 4770, 4770]]))

In [12]:
x_pos = np.ones((len(start_ix), block_size, 3), dtype=np.int64) * 0
for i, (s, e) in enumerate(zip(start_ix, end_ix)):
    src_start = max(0, s)
    src_end = e
    dst_start = max(0, -s)
    dst_end = block_size
    x_pos[i, dst_start:dst_end, :] = positions[src_start:src_end, :]

In [32]:
x_pos.shape, x_pos

(torch.Size([8, 12288, 3]),
 tensor([[[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0],
          ...,
          [ 1, 29, 47],
          [ 1, 29, 48],
          [ 1, 29, 49]],
 
         [[ 4,  1, 18],
          [ 4,  1, 19],
          [ 4,  1, 20],
          ...,
          [ 7,  1, 15],
          [ 7,  1, 16],
          [ 7,  1, 17]],
 
         [[ 3, 17, 20],
          [ 3, 17, 21],
          [ 3, 17, 22],
          ...,
          [ 6, 17, 17],
          [ 6, 17, 18],
          [ 6, 17, 19]],
 
         ...,
 
         [[ 2,  6,  6],
          [ 2,  6,  7],
          [ 2,  6,  8],
          ...,
          [ 5,  6,  3],
          [ 5,  6,  4],
          [ 5,  6,  5]],
 
         [[ 0, 38, 54],
          [ 0, 38, 55],
          [ 0, 38, 56],
          ...,
          [ 3, 38, 51],
          [ 3, 38, 52],
          [ 3, 38, 53]],
 
         [[ 1, 53, 26],
          [ 1, 53, 27],
          [ 1, 53, 28],
          ...,
          [ 4, 53, 23],
          [ 4, 53, 24],
          [

In [13]:
y = torch.stack(
    [
        torch.from_numpy((
            data2[end_irow[i], [e]]
        ).astype(np.int64))
        for i, e in enumerate(end_ix)
    ]
)
print(y.shape)
y

torch.Size([8, 1])


tensor([[4770],
        [4770],
        [4741],
        [4770],
        [4770],
        [4770],
        [4770],
        [4770]])

In [14]:
y_pos = torch.stack(
    [
        torch.from_numpy((
            positions[[e]]
        ).astype(np.int64))
        for e in end_ix
    ]
).squeeze(1)
print(x_pos.shape)
print(x.shape)
print(y_pos.shape)
print(y.shape)

(8, 12288, 3)
(8, 12288)
torch.Size([8, 3])
torch.Size([8, 1])


In [34]:
y_pos

tensor([[ 1, 29, 50],
        [ 7,  1, 18],
        [ 6, 17, 20],
        [ 3, 33,  8],
        [ 3,  4,  9],
        [ 5,  6,  6],
        [ 3, 38, 54],
        [ 4, 53, 26]])

In [15]:
x = torch.from_numpy(x) if not isinstance(x, torch.Tensor) else x
x_pos = torch.from_numpy(x_pos) if not isinstance(x_pos, torch.Tensor) else x_pos

## Debug the forward() function

In [16]:

idx_to_ignore = (x == IGNORE_TOKEN)
#  to the loss function to ignore these tokens
relative_pos = y_pos.unsqueeze(1) - x_pos
euclidean_dist = torch.norm(relative_pos.float(), dim=-1, keepdim=False)
euclidean_dist_exp = torch.exp(-euclidean_dist)
y_pos.shape, x_pos.shape, relative_pos.shape

(torch.Size([8, 3]), torch.Size([8, 12288, 3]), torch.Size([8, 12288, 3]))

In [17]:
x_pos[0][-3:, :], y_pos[0], relative_pos[0][-3:, :]

(tensor([[ 1, 29, 47],
         [ 1, 29, 48],
         [ 1, 29, 49]]),
 tensor([ 1, 29, 50]),
 tensor([[0, 0, 3],
         [0, 0, 2],
         [0, 0, 1]]))

In [18]:
euclidean_dist_exp[0]

tensor([7.8239e-26, 7.8239e-26, 7.8239e-26,  ..., 4.9787e-02, 1.3534e-01,
        3.6788e-01])

In [19]:
euclidean_dist_exp[euclidean_dist_exp < 1e-6] = 0
euclidean_dist_exp += torch.rand_like(euclidean_dist_exp) * 1e-6

In [20]:
idx_to_ignore.shape, euclidean_dist_exp.shape

(torch.Size([8, 12288]), torch.Size([8, 12288]))

In [21]:
euclidean_dist_exp[idx_to_ignore] = -1
sorted_idx = torch.argsort(euclidean_dist_exp, dim=1, descending=True)
x_pos.shape, x.shape, sorted_idx.shape

(torch.Size([8, 12288, 3]), torch.Size([8, 12288]), torch.Size([8, 12288]))

In [22]:
START_TOKEN = 5122
x_reordered = torch.gather(x, 1, sorted_idx)
x_pos_reordered = torch.gather(x_pos, 1, sorted_idx.unsqueeze(-1).expand(-1, -1, x_pos.shape[-1]))
relative_pos_reordered = torch.gather(relative_pos, 1, sorted_idx.unsqueeze(-1).expand(-1, -1, relative_pos.shape[-1]))
# re-order
x_reordered = torch.cat([torch.ones_like(x_reordered[:, :1]) * START_TOKEN, x_reordered], dim=1)
x_pos_reordered = torch.cat([torch.zeros_like(x_pos_reordered[:, :1]), x_pos_reordered], dim=1)
relative_pos_reordered = torch.cat([torch.zeros_like(relative_pos_reordered[:, :1]), relative_pos_reordered], dim=1)

In [23]:
y_pos[0], x_pos_reordered[0][:5, :]

(tensor([ 1, 29, 50]),
 tensor([[ 0,  0,  0],
         [ 1, 29, 49],
         [ 0, 29, 50],
         [ 1, 28, 50],
         [ 1, 28, 49]]))

In [24]:
# prune
sparse_block_size = 120
x_pruned = x_reordered[:, : sparse_block_size]
x_pos_pruned = x_pos_reordered[:, : sparse_block_size]
relative_pos_pruned = relative_pos_reordered[:, : sparse_block_size]
idx_to_ignore = idx_to_ignore[:, : sparse_block_size]

In [25]:
# idx_to_ignore
relative_pos_pruned[0][:5, :], x_pos_pruned[0][:5, :]

(tensor([[0, 0, 0],
         [0, 0, 1],
         [1, 0, 0],
         [0, 1, 0],
         [0, 1, 1]]),
 tensor([[ 0,  0,  0],
         [ 1, 29, 49],
         [ 0, 29, 50],
         [ 1, 28, 50],
         [ 1, 28, 49]]))

## Generation

In [26]:
pipe.position_shape = data.shape[1:]
model = pipe.create_model()

Initializing a new model from scratch


number of parameters: 89.00M
MDGPT(
  (transformer): ModuleDict(
    (wte): Embedding(5140, 768)
    (wpe): ModuleList(
      (0): Embedding(17, 768)
      (1): Embedding(129, 768)
      (2): Embedding(129, 768)
    )
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate=none)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm()
        (attn): 

In [27]:
cond_x = torch.tensor([[5121]])
cond_x_pos = torch.tensor([[[0, 0, 0]]])
cond_y_pos = torch.tensor([[0, 0, 1]])
print(cond_x.shape, cond_x_pos.shape, cond_y_pos.shape)
pipe.model.to("cpu")
logits, _ = pipe.model(x_pos=cond_x_pos, x=cond_x, y_pos=cond_y_pos)

torch.Size([1, 1]) torch.Size([1, 1, 3]) torch.Size([1, 3])
relative_pos shape: torch.Size([1, 1, 3])
position_shape: (8, 64, 64)
d=0, offset=8, relative_pos_pruned shape: torch.Size([1, 2, 3])
embedding model: Embedding(17, 768)
input to embedding: tensor([[8, 8]])
d=1, offset=64, relative_pos_pruned shape: torch.Size([1, 2, 3])
embedding model: Embedding(129, 768)
input to embedding: tensor([[64, 64]])
d=2, offset=64, relative_pos_pruned shape: torch.Size([1, 2, 3])
embedding model: Embedding(129, 768)
input to embedding: tensor([[64, 65]])
after linear, x shape: torch.Size([1, 2, 768])


In [28]:
pipe.model.generate(
    # repeat along the first axis
    cond_x_pos.repeat(2, 1, 1),
    cond_x.repeat(2, 1),
    cond_y_pos.repeat(2, 1).unsqueeze(1),
)

relative_pos shape: torch.Size([2, 1, 3])
position_shape: (8, 64, 64)
d=0, offset=8, relative_pos_pruned shape: torch.Size([2, 2, 3])
embedding model: Embedding(17, 768)
input to embedding: tensor([[8, 8],
        [8, 8]])
d=1, offset=64, relative_pos_pruned shape: torch.Size([2, 2, 3])
embedding model: Embedding(129, 768)
input to embedding: tensor([[64, 64],
        [64, 64]])
d=2, offset=64, relative_pos_pruned shape: torch.Size([2, 2, 3])
embedding model: Embedding(129, 768)
input to embedding: tensor([[64, 65],
        [64, 65]])
after linear, x shape: torch.Size([2, 2, 768])


tensor([[5121, 4234],
        [5121, 4624]])